# Notebook 01: Statistical Page Classification (Layer 1)

This notebook demonstrates the 6-signal statistical classifier that determines
whether each page is machine-readable, scanned, or hybrid.

**Signals:**
1. Font Presence (0.30)
2. Text Density (0.20)
3. Image Coverage (0.15)
4. Text Encoding Quality (0.15)
5. PDF Operator Ratio (0.10)
6. OCR Confidence (0.10) -- only for ambiguous pages

In [ ]:
import sys
sys.path.insert(0, '..')

import fitz
import yaml
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from src.classifier.page_classifier import PageClassifier
from src.classifier.region_classifier import RegionClassifier
from src.classifier.signals import (
    score_font_presence,
    score_text_density,
    score_image_coverage,
    score_text_encoding_quality,
    score_pdf_operators,
)

with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

classifier = PageClassifier(config)
region_classifier = RegionClassifier(config)
print('Classifier loaded. Signal weights:', classifier.weights)

## Load Test PDF

In [ ]:
PDF_PATH = 'sample_pdfs/v1.pdf'  # Update with your PDF

doc = fitz.open(PDF_PATH)
print(f'Document: {PDF_PATH}, Pages: {doc.page_count}')

## Classify All Pages

Run the classifier on every page and collect results.

In [ ]:
classifications = classifier.classify_document(doc, run_ocr_signal=False)

# Build results DataFrame
rows = []
for cls in classifications:
    scores = cls.signal_scores.as_dict()
    rows.append({
        'page': cls.page_number,
        'type': cls.overall_type,
        'confidence': round(cls.confidence, 3),
        **{k: round(v, 3) for k, v in scores.items()},
    })

df_results = pd.DataFrame(rows)
print(df_results.to_string(index=False))

## Visualize Signal Distributions

In [ ]:
signal_cols = ['font_presence', 'text_density', 'image_coverage', 'text_quality', 'operator_ratio', 'ocr_confidence']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, col in enumerate(signal_cols):
    ax = axes[i]
    weight = classifier.weights.get(col, 0)
    colors = ['green' if t == 'machine_readable' else 'red' if t == 'scanned' else 'orange'
              for t in df_results['type']]
    ax.bar(df_results['page'], df_results[col], color=colors, alpha=0.7)
    ax.set_title(f'{col} (weight={weight})')
    ax.set_xlabel('Page')
    ax.set_ylabel('Score')
    ax.set_ylim([0, 1.1])
    ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)

plt.suptitle('Signal Scores per Page (Green=Machine, Red=Scanned, Orange=Hybrid)', fontsize=14)
plt.tight_layout()
plt.show()

## Composite Score Distribution

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

colors = {'machine_readable': 'green', 'scanned': 'red', 'hybrid': 'orange'}
bar_colors = [colors[t] for t in df_results['type']]

ax.bar(df_results['page'], df_results['confidence'], color=bar_colors, alpha=0.8, edgecolor='black')
ax.axhline(y=classifier.thresh_machine, color='green', linestyle='--', label=f'Machine threshold ({classifier.thresh_machine})')
ax.axhline(y=classifier.thresh_scanned, color='red', linestyle='--', label=f'Scanned threshold ({classifier.thresh_scanned})')
ax.set_xlabel('Page Number')
ax.set_ylabel('Composite Score')
ax.set_title('Page Classification: Composite Score')
ax.legend()
ax.set_ylim([0, 1.1])
plt.tight_layout()
plt.show()

# Summary
type_counts = df_results['type'].value_counts()
print(f'\nClassification Summary:')
for t, count in type_counts.items():
    print(f'  {t}: {count} pages')

## Individual Signal Deep-Dive

Inspect a specific page's signals in detail.

In [ ]:
INSPECT_PAGE = 0  # Change to inspect different pages

page = doc[INSPECT_PAGE]
cls = classifications[INSPECT_PAGE]

print(f'=== Page {INSPECT_PAGE} Deep Dive ===')
print(f'Overall: {cls.overall_type} (confidence: {cls.confidence:.3f})')
print(f'\nSignal Breakdown:')

scores = cls.signal_scores.as_dict()
for signal, score in scores.items():
    weight = classifier.weights.get(signal, 0)
    contribution = weight * score
    print(f'  {signal:25s}: score={score:.3f}  x weight={weight:.2f}  = {contribution:.4f}')

print(f'\nPage info:')
print(f'  Text extracted: {len(page.get_text("text").strip())} chars')
print(f'  Fonts: {len(page.get_fonts(full=True))}')
print(f'  Images: {len(page.get_images(full=True))}')

## Region-Level Classification (for Hybrid Pages)

For pages classified as 'hybrid', segment into regions and classify each.

In [ ]:
import cv2

# Find hybrid pages (or pick any page to demo)
hybrid_pages = [cls for cls in classifications if cls.is_hybrid]
demo_page_num = hybrid_pages[0].page_number if hybrid_pages else 0

page = doc[demo_page_num]
pix = page.get_pixmap(dpi=150)
page_image = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
if pix.n == 4:
    page_image = cv2.cvtColor(page_image, cv2.COLOR_RGBA2BGR)
elif pix.n == 3:
    page_image = cv2.cvtColor(page_image, cv2.COLOR_RGB2BGR)

# Run region classifier
regions = region_classifier.classify_regions(page, page_image)

print(f'Page {demo_page_num}: Found {len(regions)} regions')
for i, r in enumerate(regions):
    print(f'  Region {i}: {r.region_type.value} ({r.extraction_strategy}) '
          f'confidence={r.confidence:.2f} bbox={tuple(int(x) for x in r.bbox)}')

In [ ]:
# Visualize regions with bounding boxes
vis_image = page_image.copy()
color_map = {
    'native_text': (0, 200, 0),     # Green
    'scanned_text': (0, 0, 200),    # Red
    'native_table': (200, 200, 0),  # Cyan
    'scanned_table': (0, 100, 200), # Orange
    'image': (200, 0, 200),         # Magenta
    'drawing': (200, 100, 0),       # Blue
}

scale_x = pix.w / page.rect.width
scale_y = pix.h / page.rect.height

for r in regions:
    x0, y0, x1, y1 = r.bbox
    px0 = int(x0 * scale_x)
    py0 = int(y0 * scale_y)
    px1 = int(x1 * scale_x)
    py1 = int(y1 * scale_y)
    color = color_map.get(r.region_type.value, (128, 128, 128))
    cv2.rectangle(vis_image, (px0, py0), (px1, py1), color, 3)
    label = f'{r.region_type.value} ({r.extraction_strategy})'
    cv2.putText(vis_image, label, (px0, py0 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

plt.figure(figsize=(12, 16))
plt.imshow(cv2.cvtColor(vis_image, cv2.COLOR_BGR2RGB))
plt.title(f'Page {demo_page_num} - Region Classification')
plt.axis('off')
plt.show()

In [ ]:
doc.close()
print('Done! Classification pipeline validated.')